# Procesamiento

## Previo: cargar librería de CSV

Para poder cargar ficheros CSV en Spark, es necesario haber arrancado la shell con soporte adecuado, o haber desplegado el JAR en los workers del cluster.

Desde la shell el comando sería:

*spark-shell --packages com.databricks:spark-csv_2.11:1.2.0*

En nuestro caso, hemos optado por añadir lo siguiente al fichero de configuración de Spark, */usr/local/spark/conf/spark-defaults.conf*:
  
    spark.jars.packages     com.databricks:spark-csv_2.10:1.3.0


In [1]:
val blue_box_2 = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").load("datos/blue_box_2.csv")


## Cargar vértices y aristas en un *GraphRDD*

In [3]:
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import scala.util.MurmurHash

val linksToFrom = blue_box_2.select("Source", "Target")
val pageNames = blue_box_2.select("Source", "Target").flatMap(x => Iterable(x(0).toString, x(1).toString))
val pageVertices: RDD[(VertexId, String)] = pageNames.distinct().map(x => (MurmurHash.stringHash(x), x))
val linkEdges = linksToFrom.map(x => ((MurmurHash.stringHash(x(0).toString),MurmurHash.stringHash(x(1).toString)), 1)).reduceByKey(_+_).map(x => Edge(x._1._1, x._1._2,x._2))
val graph = Graph(pageVertices, linkEdges)
graph.numVertices // 475
graph.numEdges // 499

499